In [1]:
import os
os.environ['FOR_DISABLE_CONSOLE_CTRL_HANDLER'] = '1'

import numpy as np
import pandas as pd
import wandb
import torch
import argparse
from tqdm import tqdm
from torch.optim import AdamW
from datetime import datetime
from sentence_transformers import losses, SentenceTransformer
from types import SimpleNamespace

from modules.ModelFunctions import get_ST_model, auto_load_model
from modules.timed_logger import logger
from modules.metrics import evaluate_embedding_similarity_with_mrr
from modules.STHardNegMiner import mine_negatives, _pairs_to_dataset
from sentence_transformers.util import mine_hard_negatives

logger.reset_timer()

In [2]:

args = {
    "test_mode": False,
    
    "no_relation": False,  # Disable relation data even if files exist or config says True
    "range_min": 10,       # Minimum rank for candidate negatives
    "range_max": 50,       # Maximum rank for candidate negatives
    "relative_margin": 0.01,  # Relative margin for mining
    "num_neg_matching": None,  # Negatives per anchor for matching mining (override)
    "num_neg_relation": None,  # Negatives per anchor for relation mining (override)
    "sampling_strategy": "top",  # Negative sampling strategy from candidates
    "mine_batch_size": 256,      # Batch size for miner embedding
    "no_faiss": False,           # Disable FAISS acceleration in miner
    "model_checkpoint": "none",  # Path to model checkpoint to load (default: none - use HF base model)
    
    
    "use_relation": True,
    "n_pos_matching": 5,
    "n_neg_matching": 5,
    "n_fp_matching": 5,
    "n_pos_relation": 5,
    "n_neg_relation": 5,
    "n_fp_relation": 5,
    
}

args = SimpleNamespace(**args)


# Initialize Model

In [3]:

logger.log("Loading Model (pos-aware training)")
output_dir = f"output/finetune_posaware/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
base_model = 'all-MiniLM-L6-v2'

# Load model - use Hugging Face model directly or specified checkpoint
if args.model_checkpoint and args.model_checkpoint != "none":
    model, tokenizer, args = auto_load_model(args.model_checkpoint)
else:
    model, tokenizer = None, None

if model is None:
    # Load base ST model with special tokens for relation support
    logger.log(f"Loading base model with special tokens: {base_model}")
    model, tokenizer = get_ST_model(base_model)

# start_epoch = args.epoch
# start_batch_i = args.batch_i + 1
# start_global_step = args.global_step + 1

2025-10-01 23:23:45 - Loading Model (pos-aware training) (Elapsed: 0.03s, Since last: 0.03s)
2025-10-01 23:23:45 - Loading base model with special tokens: all-MiniLM-L6-v2 (Elapsed: 0.04s, Since last: 0.00s)
2025-10-01 23:23:45 - Use pytorch device_name: cuda:0
2025-10-01 23:23:45 - Load pretrained SentenceTransformer: models/all-MiniLM-L6-v2_ST_childof_parentof\auto_save_1_20251001_164241


Loaded latest auto-saved model from: models/all-MiniLM-L6-v2_ST_childof_parentof\auto_save_1_20251001_164241


# Load OMOP data

In [4]:
logger.log("Loading training data")

matching_base_path = "data/matching"
relation_base_path = "data/relation"
seed = 42


target_concepts_path = os.path.join(matching_base_path, 'target_concepts.feather')
target_concepts = pd.read_feather(target_concepts_path)
matching_name_bridge = pd.read_feather(os.path.join(matching_base_path, 'condition_matching_name_bridge_train.feather'))
matching_name_table = pd.read_feather(os.path.join(matching_base_path, 'condition_matching_name_table_train.feather'))

target_concepts.shape, matching_name_bridge.shape, matching_name_table.shape

2025-10-01 23:23:45 - Loading training data (Elapsed: 0.72s, Since last: 0.68s)


((160288, 2), (643319, 2), (566536, 5))

# Training dataset creation

In [5]:


# Build anchor/positive pairs for matching and mine negatives with ST builtin
anchor_positive_match = matching_name_bridge
anchor_positive_match = anchor_positive_match.merge(
    matching_name_table[['name_id','name']].rename(columns={'name':'positive'}),
    on='name_id'
).merge(
    target_concepts[['concept_id','concept_name']].rename(columns={'concept_name':'anchor'}),
    on='concept_id'
)[['anchor','positive']].drop_duplicates()

if args.test_mode:
    anchor_positive_match = anchor_positive_match.iloc[:1000]


matching_pos_ds = _pairs_to_dataset(anchor_positive_match)

matching_fp_df = mine_hard_negatives(
    dataset=matching_pos_ds,
    model=model,
    range_min=args.range_min,
    range_max=args.range_max,
    relative_margin=args.relative_margin,
    num_negatives=args.n_fp_matching,
    sampling_strategy=args.sampling_strategy,
    batch_size=args.mine_batch_size,
    use_faiss=not args.no_faiss,
    verbose=True,
)

matching_fp_df


Found 104164 unique queries out of 643269 total queries.
Found an average of 6.176 positives per query.


Batches:   0%|          | 0/2214 [00:00<?, ?it/s]

c:\Users\jiewang\AppData\Local\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Batches:   0%|          | 0/407 [00:00<?, ?it/s]

2025-10-01 23:24:52 - Loading faiss with AVX512 support.
2025-10-01 23:24:52 - Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
2025-10-01 23:24:52 - Loading faiss with AVX2 support.
2025-10-01 23:24:52 - Successfully loaded faiss with AVX2 support.
2025-10-01 23:24:52 - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.
Querying FAISS index: 100%|██████████| 7/7 [03:13<00:00, 27.59s/it]


Negative candidates mined, preparing dataset...
Metric       Positive       Negative     Difference
Count         643,269        616,354               
Mean           0.7081         0.7353         0.1659
Median         0.7682         0.7429         0.1752
Std            0.2435         0.0945         0.1289
Min           -0.1426         0.3179        -0.8590
25%            0.5470         0.6772         0.1049
50%            0.7682         0.7429         0.1752
75%            0.9215         0.8024         0.2431
Max            1.0000         0.9717         0.6315
Skipped 2,042,592 potential negatives (38.45%) due to the relative_margin of 0.01.
Could not find enough negatives for 2599991 samples (80.84%). Consider adjusting the range_max, range_min and relative_margin parameters if you'd like to find more valid negatives.


Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 616354
})

In [6]:
matching_fp_df[0]

{'anchor': 'X-linked lethal multiple pterygium syndrome',
 'positive': 'A rare genetic developmental defect during embryogenesis. The disease has the typical lethal multiple pterygium syndrome presentation comprising of multiple pterygia, severe arthrogryposis, cleft palate, cystic hygromata and/or fetal hydrops, skeletal abnormalities and fetal death in the second or third trimester with an X-linked pattern of inheritance.',
 'negative': 'A group of rare genetic disorders characterized by the presence of joint contractures and multiple soft tissue webs (pterygia) across the neck and various joints, as well as typical facial appearance and a variety of other congenital anomalies. Both lethal (lethal and X-linked lethal multiple pterygium syndrome) and non-lethal (autosomal recessive and autosomal dominant multiple pterygium syndrome) forms occur.'}

In [7]:

relation_table_path = os.path.join(relation_base_path, 'name_table_relation.feather')
relation_bridge_path = os.path.join(relation_base_path, 'name_bridge_relation.feather')
relation_available = os.path.exists(relation_table_path) and os.path.exists(relation_bridge_path)
if args.use_relation and relation_available:
    name_table_relation = pd.read_feather(relation_table_path)
    name_bridge_relation = pd.read_feather(relation_bridge_path)


    anchor_positive_rel = name_bridge_relation
    anchor_positive_rel = anchor_positive_rel.merge(
        name_table_relation[['name_id','name']].rename(columns={'name':'positive'}),
        on='name_id'
    ).merge(
        target_concepts[['concept_id','concept_name']].rename(columns={'concept_name':'anchor'}),
        on='concept_id'
    )[['anchor','positive']].drop_duplicates()

    
    if args.test_mode:
        anchor_positive_rel = anchor_positive_rel.iloc[:1000]
    
    relation_pos_ds = _pairs_to_dataset(anchor_positive_rel)
    
    relation_fp_df = mine_hard_negatives(
        dataset=relation_pos_ds,
        model=model,
        range_min=args.range_min,
        range_max=args.range_max,
        relative_margin=args.relative_margin,
        num_negatives=args.n_fp_relation,
        sampling_strategy=args.sampling_strategy,
        batch_size=args.mine_batch_size,
        use_faiss=not args.no_faiss,
        verbose=True,
    )


relation_fp_df

Found 42095 unique queries out of 1029692 total queries.
Found an average of 24.461 positives per query.


Batches:   0%|          | 0/624 [00:00<?, ?it/s]

Batches:   0%|          | 0/165 [00:00<?, ?it/s]

Querying FAISS index: 100%|██████████| 3/3 [00:22<00:00,  7.62s/it]


Negative candidates mined, preparing dataset...
Metric       Positive       Negative     Difference
Count       1,029,692        216,216               
Mean           0.5357         0.6601         0.0993
Median         0.5383         0.6670         0.1182
Std            0.1937         0.0940         0.1440
Min           -0.1423         0.2245        -0.7102
25%            0.3903         0.6030         0.0456
50%            0.5383         0.6670         0.1182
75%            0.6841         0.7262         0.1890
Max            0.9875         0.9574         0.5915
Skipped 1,181,225 potential negatives (55.02%) due to the relative_margin of 0.01.
Could not find enough negatives for 4932244 samples (95.80%). Consider adjusting the range_max, range_min and relative_margin parameters if you'd like to find more valid negatives.


Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 216216
})

## Combine and Create final training dataset

In [8]:
from datasets import concatenate_datasets

if args.use_relation and relation_available:
    ds_all = concatenate_datasets([matching_fp_df, relation_fp_df])
else:
    ds_all = matching_fp_df
    
ds_all

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 832570
})

# Validation dataset

In [9]:
logger.log("Loading validation data")
condition_matching_valid = pd.read_feather(os.path.join(matching_base_path, 'condition_matching_valid.feather'))
print(condition_matching_valid.columns)

condition_matching_train_subset = pd.read_feather(os.path.join(matching_base_path, 'condition_matching_train_subset.feather'))
print(condition_matching_train_subset.columns)

condition_relation_train_subset = pd.read_feather(os.path.join(relation_base_path, 'condition_relation_train_subset.feather'))
print(condition_relation_train_subset.columns)

2025-10-01 23:30:54 - Loading validation data (Elapsed: 429.53s, Since last: 428.81s)


Index(['corpus_name', 'query_name', 'corpus_id', 'query_id', 'label'], dtype='object')
Index(['query_name', 'corpus_name', 'query_id', 'corpus_id', 'label'], dtype='object')
Index(['query_name', 'corpus_name', 'query_id', 'corpus_id', 'label'], dtype='object')


# Model Training settings

In [10]:
from sentence_transformers.evaluation import SentenceEvaluator
# Create a custom evaluator using your existing evaluation function
class CustomMRREvaluator(SentenceEvaluator):
    def __init__(self, validation_data, name="custom_eval"):
        self.validation_data = validation_data
        self.name = name
        
    def __call__(self, model, output_path=None, epoch=None, steps=None):
        # Use your existing evaluation function
        eval_results = evaluate_embedding_similarity_with_mrr(model, self.validation_data)
        
        # Return the main metric (MRR) for model selection
        eval_results = {f"{self.name}/{k}": v for k, v in eval_results.items()}
        return eval_results
    

# Create custom evaluator using your existing function
evaluator_valid = CustomMRREvaluator(condition_matching_valid, name="eval_valid")
evaluator_train_matching = CustomMRREvaluator(condition_matching_train_subset, name="train_matching")
evaluator_train_relation = CustomMRREvaluator(condition_relation_train_subset, name="train_relation")

evaluator_valid(model)

Batches:   0%|          | 0/84 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

{'eval_valid/roc_auc': 0.7680990833089902,
 'eval_valid/average_precision': 0.4821933667885799,
 'eval_valid/f1_score': 0.44631901840490795,
 'eval_valid/precision': 0.36014851485148514,
 'eval_valid/recall': 0.5866935483870968,
 'eval_valid/accuracy': 0.7435168738898756,
 'eval_valid/best_hit1': 0.7157258064516129,
 'eval_valid/best_hit3': 0.8487903225806451,
 'eval_valid/best_hit5': 0.9153225806451613,
 'eval_valid/best_hit10': 1.0,
 'eval_valid/best_hit20': 1.0,
 'eval_valid/best_hit50': 1.0,
 'eval_valid/best_hit100': 1.0,
 'eval_valid/best_reciprocal_rank': 0.802284946236559,
 'eval_valid/worst_hit1': 0.7157258064516129,
 'eval_valid/worst_hit3': 0.8487903225806451,
 'eval_valid/worst_hit5': 0.9153225806451613,
 'eval_valid/worst_hit10': 1.0,
 'eval_valid/worst_hit20': 1.0,
 'eval_valid/worst_hit50': 1.0,
 'eval_valid/worst_hit100': 1.0,
 'eval_valid/worst_reciprocal_rank': 0.802284946236559}

In [11]:
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments

max_saves = 4

epoch_num = 10
batch_size = 256

wandb_report_steps = 256
arg_eval_steps = 2048
arg_saving_steps = arg_eval_steps * 2

learning_rate = 2e-5


loss_func = losses.TripletLoss(model=model)
optimizer = AdamW(model.parameters(), lr=learning_rate)


# Create training arguments equivalent to your custom training loop
training_args = SentenceTransformerTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=epoch_num,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    fp16=False,
    bf16=False,
    learning_rate=learning_rate,
    eval_strategy="steps",
    eval_steps=arg_eval_steps,
    save_strategy="steps",
    save_steps=arg_saving_steps,
    save_total_limit=max_saves,
    logging_steps=wandb_report_steps,
    run_name=output_dir,
    seed=seed,
    data_seed=seed,
    report_to="wandb" if not args.test_mode else "none",
    remove_unused_columns=False,
    dataloader_drop_last=False,
    dataloader_num_workers=0,  # Avoid multiprocessing issues on Windows
    metric_for_best_model="eval_valid/roc_auc",
    greater_is_better=True,
    load_best_model_at_end=True,
)



In [12]:
# Initialize the trainer
trainer = SentenceTransformerTrainer(
    model=model,
    args=training_args,
    train_dataset=ds_all,
    eval_dataset=None,  # Using custom evaluator instead
    loss=loss_func,
    evaluator=[evaluator_valid, evaluator_train_matching, evaluator_train_relation],
    tokenizer=tokenizer,
)

# Start training
logger.log(f"Starting SentenceTransformer training. Model saved to: {output_dir}")
trainer.train(resume_from_checkpoint=args.model_checkpoint if args.model_checkpoint != "none" else None)

# Save the final model
trainer.save_model()
logger.log(f"Training completed. Model saved to {output_dir}")
logger.done()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

2025-10-01 23:30:57 - Starting SentenceTransformer training. Model saved to: output/finetune_posaware/2025-10-01_23-23-45 (Elapsed: 432.34s, Since last: 2.81s)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: szwjf08 (szwjf08-utmb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  0%|          | 0/32530 [00:00<?, ?it/s]

{'loss': 5.1504, 'grad_norm': 0.9686791300773621, 'learning_rate': 1.5739317553027975e-06, 'epoch': 0.08}
{'loss': 4.8542, 'grad_norm': 0.9927756786346436, 'learning_rate': 3.147863510605595e-06, 'epoch': 0.16}
{'loss': 4.6768, 'grad_norm': 4.178518772125244, 'learning_rate': 4.721795265908392e-06, 'epoch': 0.24}
{'loss': 4.5962, 'grad_norm': 4.067861557006836, 'learning_rate': 6.29572702121119e-06, 'epoch': 0.31}
{'loss': 4.5725, 'grad_norm': 4.412888526916504, 'learning_rate': 7.869658776513987e-06, 'epoch': 0.39}
{'loss': 4.5584, 'grad_norm': 9.086004257202148, 'learning_rate': 9.443590531816785e-06, 'epoch': 0.47}
{'loss': 4.5384, 'grad_norm': 2.9517104625701904, 'learning_rate': 1.1017522287119582e-05, 'epoch': 0.55}
{'loss': 4.5402, 'grad_norm': 10.846002578735352, 'learning_rate': 1.259145404242238e-05, 'epoch': 0.63}


Batches:   0%|          | 0/84 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

{'eval_valid/roc_auc': 0.5728671111018376, 'eval_valid/average_precision': 0.20722154054582115, 'eval_valid/f1_score': 0.3029733959311424, 'eval_valid/precision': 0.1793256761763616, 'eval_valid/recall': 0.9758064516129032, 'eval_valid/accuracy': 0.2088809946714032, 'eval_valid/best_hit1': 0.20766129032258066, 'eval_valid/best_hit3': 0.5201612903225806, 'eval_valid/best_hit5': 0.8709677419354839, 'eval_valid/best_hit10': 1.0, 'eval_valid/best_hit20': 1.0, 'eval_valid/best_hit50': 1.0, 'eval_valid/best_hit100': 1.0, 'eval_valid/best_reciprocal_rank': 0.43887768817204303, 'eval_valid/worst_hit1': 0.20766129032258066, 'eval_valid/worst_hit3': 0.5201612903225806, 'eval_valid/worst_hit5': 0.8709677419354839, 'eval_valid/worst_hit10': 1.0, 'eval_valid/worst_hit20': 1.0, 'eval_valid/worst_hit50': 1.0, 'eval_valid/worst_hit100': 1.0, 'eval_valid/worst_reciprocal_rank': 0.43887768817204303, 'eval_train_matching/roc_auc': 0.5269596162480895, 'eval_train_matching/average_precision': 0.21001253378

2025-10-02 02:31:48 - Saving model checkpoint to output/finetune_posaware/2025-10-01_23-23-45\checkpoint-32530
2025-10-02 02:31:48 - Save model to output/finetune_posaware/2025-10-01_23-23-45\checkpoint-32530
2025-10-02 02:31:48 - Loading best model from output/finetune_posaware/2025-10-01_23-23-45\checkpoint-4096 (score: 0.6014419799969397).
2025-10-02 02:31:48 - Use pytorch device_name: cuda:0
2025-10-02 02:31:48 - Load pretrained SentenceTransformer: output/finetune_posaware/2025-10-01_23-23-45\checkpoint-4096


{'train_runtime': 10851.5586, 'train_samples_per_second': 767.235, 'train_steps_per_second': 2.998, 'train_loss': 4.426937723379666, 'epoch': 10.0}


2025-10-02 02:31:49 - Saving model checkpoint to output/finetune_posaware/2025-10-01_23-23-45
2025-10-02 02:31:49 - Save model to output/finetune_posaware/2025-10-01_23-23-45
2025-10-02 02:31:49 - Training completed. Model saved to output/finetune_posaware/2025-10-01_23-23-45 (Elapsed: 11284.57s, Since last: 10852.23s)
2025-10-02 02:31:49 - All Jobs Done! (Elapsed: 11284.57s, Since last: 0.00s)
